# Ejercicio Creativo: Tu Nombre de Rapero Generado por IA

### Generador de Nombres de Rapero
En este ejercicio vamos a jugar con los números de los tokens.
La idea es tomar tu información personal, convertirla en tokens, ordenarlos y aplicar una fórmula matemática "mística" para descubrir cuál es tu nombre artístico según GPT-4.

1.  **Entrada:** Definimos tus datos.
2.  **Procesamiento:** Convertimos todo a tokens y los ordenamos.
3.  **Algoritmo:**
    * Parte 1: Basada en la primera letra de tu nombre.
    * Parte 2: Basada en la última letra de tu apellido.
    * Parte 3: Una mezcla matemática de tus tokens (el mayor dividido por el menor más el promedio de los del medio).

In [1]:
import numpy as np
import re
# Aseguramos que tiktoken esté instalado e importado
try:
    import tiktoken
except ImportError:
    !pip install tiktoken
    import tiktoken

# Inicializamos el tokenizador
tokenizer = tiktoken.get_encoding('cl100k_base')

# --- 1. DATOS DE ENTRADA ---
# Puedes cambiar esto por tus datos reales
first_name = "Michael"
last_name = "Jordan"
fav_color = "Red"

# Unimos todo en un solo texto
text = ' '.join([first_name, last_name, fav_color])
print(f"Texto base: '{text}'")

# --- 2. TOKENIZACIÓN Y ORDENAMIENTO ---
# Codificamos el texto a números (tokens)
idx = tokenizer.encode(text)

# Ordenamos la lista de tokens de menor a mayor
idx.sort()
print(f"Tokens ordenados: {idx}")

# --- 3. FUNCIÓN DE LIMPIEZA ---
# Esta función nos ayuda a evitar tokens "raros" (símbolos, espacios vacíos, bytes extraños)
# Devuelve True si el token NO es texto normal (alfanumérico)
def not_normal(token_str):
    pattern = re.compile(r'^[A-Za-z0-9]+$') # Solo letras y números
    return not bool(pattern.fullmatch(token_str.strip()))

# --- 4. CÁLCULO DEL NOMBRE DE RAPERO ---

# PARTE 1: Primera letra del nombre - 1
# Tomamos el token de la primera letra
first_letter_token = tokenizer.encode(first_name[0])[0]
part1 = first_letter_token - 1

# Si el resultado es un símbolo raro, seguimos restando hasta hallar una letra/número normal
while not_normal(tokenizer.decode([part1])):
    part1 -= 1 # Vamos hacia atrás en el vocabulario

# PARTE 2: Última letra del apellido + 1
last_letter_token = tokenizer.encode(last_name[-1])[0]
part2 = last_letter_token + 1

# Opcional: Limpiamos también la parte 2 si sale rara (sumando)
while not_normal(tokenizer.decode([part2])):
    part2 += 1

# PARTE 3: (Último token / Primero) + Promedio del medio
# Usamos los tokens ordenados en 'idx'
# idx[-1] es el más grande, idx[0] el más pequeño
# idx[1:-1] son todos los del medio
val_part3 = (idx[-1] / idx[0]) + np.mean(idx[1:-1])
part3 = int(val_part3) # Convertimos a entero para que sea un token válido

# --- 5. RESULTADO FINAL ---
rapper_tokens = [part1, part2, part3]
rapper_name = tokenizer.decode(rapper_tokens)

print("-" * 20)
print(f'Tokens generados: {rapper_tokens}')
print(f'My rapper name is "{rapper_name}"')
print("-" * 20)

Texto base: 'Michael Jordan Red'
Tokens ordenados: [3816, 17527, 26597]
--------------------
Tokens generados: [43, 78, 17533]
My rapper name is "Lo Dub"
--------------------


### Explicación del Código

* **`idx.sort()`:** Al ordenar los tokens, desordenamos el significado original de las palabras, quedándonos solo con sus valores numéricos para hacer "numerología".
* **`not_normal()`:** El vocabulario de GPT-4 tiene muchos tokens extraños (símbolos chinos, bytes raw, emojis). Este bucle `while` asegura que tu nombre sea legible forzando al algoritmo a buscar el siguiente token "normal" vecino.
* **El Resultado:** Es probable que obtengas una combinación de sílabas o palabras cortas que suenan extraño. ¡Ese es tu nombre generado por la "creatividad" matemática del tokenizador!

### Explicación del Algoritmo: "Nombre de Rapero"

Este ejercicio es un juego matemático para demostrar que, para la IA, las palabras no son más que una bolsa de números. Al manipular esos números con sumas, restas y promedios, podemos generar "nuevas palabras".

Aquí está el desglose de lo que hace el código paso a paso:

#### 1. Desordenar el Significado (`idx.sort`)
Primero, tomamos tus datos (Nombre + Apellido + Color) y los convertimos en una lista de tokens.
* **Lo normal:** Los tokens mantienen un orden para que la frase tenga sentido (Sujeto + Verbo + Predicado).
* **Lo que hacemos aquí:** Al ejecutar `idx.sort()`, destruimos el orden gramatical. Ahora solo tenemos una lista de números ordenados de menor a mayor. Tratamos al lenguaje como pura estadística.

#### 2. El Filtro de Limpieza (`not_normal`)
El vocabulario de GPT-4 (`cl100k_base`) contiene ~100,000 tokens. Muchos de ellos no son palabras, sino símbolos raros, espacios vacíos, emojis o fragmentos de código.
* Esta función verifica si un token es "normal" (letras y números) usando una expresión regular (`re`).
* Si el cálculo matemático nos da un símbolo extraño (ej: `¾` o `\n`), el bucle `while` nos obliga a seguir buscando hasta encontrar un token legible.

#### 3. La "Fórmula Mágica"
El nombre se construye uniendo 3 piezas generadas matemáticamente:

* **Parte 1 (El Prefijo):**
    * Toma la **primera letra** de tu nombre.
    * Busca su ID numérico y le **resta 1**.
    * Es decir, busca el "vecino anterior" en el diccionario de la IA.

* **Parte 2 (El Sufijo):**
    * Toma la **última letra** de tu apellido.
    * Busca su ID numérico y le **suma 1**.
    * Es decir, busca el "vecino siguiente".

* **Parte 3 (El Núcleo):**
    * Aquí usamos la lista ordenada `idx`.
    * Aplica una fórmula arbitraria: `(Token_Mayor / Token_Menor) + Promedio(Tokens_Del_Medio)`.
    * El resultado es un número nuevo que representa una mezcla matemática de todos tus datos personales.

#### Resultado Final
El código une los tres resultados `[part1, part2, part3]` y los decodifica. El resultado suele ser una palabra inventada o un sonido extraño (ej: *"Lz-qwe-Bop"*), que actúa como tu nombre artístico único generado por la "creatividad matemática" del tokenizador.